In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers datasets scikit-learn torch

  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2024.12.0-py3-none-any.whl (183 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.


In [3]:
# 步骤3：导入模块
import os
import numpy as np
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
# os.environ['WANDB_API_KEY'] = '255fab36462f5587d825c69b9d5b53a852a2c4d3'  # 替换为你的实际 API 密钥
os.environ['WANDB_MODE'] = 'disabled'

# 步骤4：配置路径
DRIVE_DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/data'
file_paths = {
    'train': os.path.join(DRIVE_DATA_PATH, 'imdb_train.npz'),
    'test': os.path.join(DRIVE_DATA_PATH, 'imdb_test.npz'),
    'val': os.path.join(DRIVE_DATA_PATH, 'imdb_val.npz')
}

# 步骤5：加载数据
def load_npz(file_path):
    data = np.load(file_path)
    return data['x'], data['y']

# 加载数据集
train_texts, train_labels = load_npz(file_paths['train'])
val_texts, val_labels = load_npz(file_paths['val'])
test_texts, test_labels = load_npz(file_paths['test'])

# 新增：打印数据集大小
print(f"Conjunto de entrenamiento: {len(train_texts)} | Conjunto de validación: {len(val_texts)} | Conjunto de prueba: {len(test_texts)}")

Conjunto de entrenamiento: 20000 | Conjunto de validación: 5000 | Conjunto de prueba: 25000


In [4]:
# 步骤6：数据预处理（修复格式问题）
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def preprocess(texts, labels):
    texts = [str(text) for text in texts]  # 确保文本是字符串
    encodings = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors='pt'
    )
    # 将标签转换为张量并合并到encodings字典中
    encodings['labels'] = torch.tensor(labels)
    return encodings

# 转换为PyTorch Dataset
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels):
        self.encodings = preprocess(texts, labels)

    def __getitem__(self, idx):
        # 返回单个样本的字典格式
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': self.encodings['labels'][idx]
        }

    def __len__(self):
        return len(self.encodings['labels'])

# 重新初始化数据集
train_dataset = IMDbDataset(train_texts, train_labels)
val_dataset = IMDbDataset(val_texts, val_labels)
test_dataset = IMDbDataset(test_texts, test_labels)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [5]:
# 步骤7：模型配置
# model_save_path = "/content/drive/MyDrive/Colab Notebooks/model/imdb"
# os.makedirs(model_save_path, exist_ok=True)


# 自动检测设备（优先GPU，否则CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 初始化新模型（每次运行都会从头载入预训练权重）
print("Inicializando un nuevo modelo...")
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2,
    attention_probs_dropout_prob=0.1  # 你可以调整这个 Dropout 值
).to(device)

# 显式验证设备
print(f"Modelo cargado en: {device}")

Inicializando un nuevo modelo...


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Modelo cargado en: cuda


In [6]:
# 步骤8：训练配置
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Colab Notebooks/data",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    num_train_epochs=5,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    save_total_limit=3,
    logging_dir=os.path.join(DRIVE_DATA_PATH, 'logs'),
    logging_steps=100
)

# 步骤9：定义评估指标（新增完整指标）
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision = precision_score(labels, preds)
    recall = recall_score(labels, preds)
    f1 = f1_score(labels, preds)
    accuracy = accuracy_score(labels, preds)
    return {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1}

In [7]:
# 步骤10：初始化Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [8]:
# 执行训练
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.195200,0.201110,0.925800,0.910845,0.944000,0.927126
2,0.112000,0.215988,0.927600,0.907393,0.952400,0.929352
3,0.047900,0.350355,0.928800,0.914861,0.945600,0.929976
4,0.032800,0.333244,0.934000,0.925157,0.944400,0.934679
5,0.010400,0.365562,0.934600,0.932352,0.937200,0.934770


TrainOutput(global_step=3125, training_loss=0.09559515528619289, metrics={'train_runtime': 9666.8532, 'train_samples_per_second': 10.345, 'train_steps_per_second': 0.323, 'total_flos': 2.6311105536e+16, 'train_loss': 0.09559515528619289, 'epoch': 5.0})

In [ ]:
# # 可选保存模型
# SAVE_MODEL = False  # 改为 True 时保存
# if SAVE_MODEL:
#     trainer.save_model(model_save_path)
#     tokenizer.save_pretrained(model_save_path)

In [9]:
# 步骤12：模型评估（新增完整指标）
test_preds = trainer.predict(test_dataset)
bert_preds = np.argmax(test_preds.predictions, axis=1)

precision = precision_score(test_labels, bert_preds)
recall = recall_score(test_labels, bert_preds)
accuracy = accuracy_score(test_labels, bert_preds)
f1 = f1_score(test_labels, bert_preds)

print("\nModelo BERT en IMDb - Métricas Completas:")
print(f"Precisión: {precision:.4f} | Recall: {recall:.4f} | Accuracy: {accuracy:.4f} | F1-Score: {f1:.4f}")


Modelo BERT en IMDb - Métricas Completas:
Precisión: 0.9340 | Recall: 0.9449 | Accuracy: 0.9391 | F1-Score: 0.9394
